# 协程
- 参考资料
    - http://python.jobbole.com/86481/
    - http://python.jobbole.com/87310/
    - https://segmentfault.com/a/1190000009781688
        
# 迭代器 
- 可迭代(Iterrable)：直接作用于for循环的变量
- 迭代器(Iterator)：不但可以作用于for循环，还可以被next调用
- list是典型的可迭代对象，但不是迭代器
- 通过isinstance判断
- iterable和itrator可以转换
    - 通过iter函数

In [2]:
# 可迭代
l = [i for i in range(10)]

# l是可迭代的，但不是迭代器
for idx in l:
    print(idx)
    
# range是个迭代器
for i in range(2):
    print(i)

0
1
2
3
4
5
6
7
8
9
0
1


In [5]:
# 判断是否可迭代
from collections import Iterable
l1 = [1,23,45,2]

print(isinstance(l1,Iterable))

import sys
from collections import Iterator
print(isinstance(l1,Iterator))


True
False


In [6]:
# iter函数
s = 'i love wangxiaojing '
print(isinstance(s,Iterable))
print(isinstance(s,Iterator))

s_iter = iter(s)
print(isinstance(s_iter,Iterable))
print(isinstance(s_iter,Iterator))

True
False
True
True


# 生成器
- generator：一边循环一边计算下一个元素的机制/算法
- 需要满足三个条件：
    - 每次调用都产生出for循环需要的下一个元素或者
    - 如果达到最后一个后，爆出stopIteration异常
    - 可以被next调用
- 如何生成一个生成器
    - 直接使用
    - 如果函数中包含yield，则这个函数就叫生成器
    - next调用函数，遇到yield返回

In [9]:
# 直接使用生成器
L = [x*x for x in range(5)]# 放在中括号中是列表生成器
g = (x*x for x in range(5))# 放在笑哭哦好中就是生成器

print(type(L))
print(type(g))

<class 'list'>
<class 'generator'>


In [10]:
# 函数案例
def odd():
    print("step 1")
    print("step 2")
    print("step 3")
    return None
odd()


step 1
step 2
step 3


In [2]:
# 生成器案例
# 在函数odd中，yield负责返回
def odd():
    print("step 1")
    yield 1
    print("step 2")
    yield 2
    print("step 3")
    yield 3
# odd()是调用生成器 
g = odd()
one = next(g)
print(one)


two = next(g)
print(two)

three = next(g)
print(three)

step 1
1
step 2
2
step 3
3


In [4]:
# for循环调用生成器
def fib(max):
    n,a,b = 0,0,1  #注意写法
    while n<max:
        print(b)
        a,b = b,a+b #注意写法 
        n +=1
        
    return "Done"
fib(5)

1
1
2
3
5


'Done'

In [5]:
# 费不纳妾数列的生成器写法
def fib(max):
    n,a,b = 0,0,1  #注意写法
    while n<max:
        yield b
        a,b = b,a+b #注意写法 
        n +=1
    # 需要注意，爆出异常的是返回值return的返回值
    return "Done"
g = fib(5)

for i in range(6):
    rst = next(g)
    print(rst)

1
1
2
3
5


StopIteration: Done

In [6]:
ge = fib(10)
'''
生成器的典型用法是在for中使用
比较常用的是典型生成器就是range
'''
for i in ge:# 在for循环中使用生成器
    print(i)

1
1
2
3
5
8
13
21
34
55


# 协程
- 历史历程
    - 3.4引入协程，用yield实现
    - 3.5引入协程语法
    - 实现的协程比较好的包有asyncio，tornado，gevent
- 从技术角度来讲，协程就是一个你可以执行的函数，或者干脆把协程理解成生成器
- 协程的实现：
    - yield返回
    - send调用
    
    
- 协程终止
    - 写成中未处理的异常会向上冒泡，传给next、函数或者send方法的调用方
    - 止协程的一种方式：发送某个哨符值，让写成推出，内置的None和Ellipsis等常量经常用作哨符值
    
- yield from
    - 调用协程为了得到返回值，协程必须正常终止
    - 生成器正常终止会发出Stop Iteration异常，异常对象的vlaue属性保存返回值
    - yield from从内部朴拙异常
    - 委派生成器
        - 包含yield from表达式的生成器
        - 委派生成器在yield from表达式处暂停，调用方可以直接把数据发给自生成器
        - 子生成器再把残生的值发给调用方
        -  子生成器在最后，解释器会抛出Stop Iteration，并且把返回值附加到异常对象上
        -

In [7]:
# 协程代码案例
def simple_coroutime():
    print('->start')
    x = yield
    print('->recived',x)
    
sc = simple_coroutime()
print(1111)
# 可以使用sc.send(None)，效果一样
next(sc)# 预激

print(2222)
sc.send('zhexiao')

1111
->start
2222
->recived zhexiao


StopIteration: 

In [8]:
# 协程的状态
def simple_coroutime(a):
    print('->start')
    b = yield  a
    print('->recived',a,b)
    
    c = yield a+b
    print('->recived',a,b,c)
    
# runc
sc = simple_coroutime(5)

aa = next(sc)
print(aa)
bb = sc.send(6)
print(bb)
cc = sc.send(7)
print(cc)


->start
5
->recived 5 6
11
->recived 5 6 7


StopIteration: 

In [9]:
# yield from
def gen():
    for c in "AB":
        yield c
# list直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from 'AB'
    
print(list(gen_new()))

['A', 'B']
['A', 'B']
